<a href="https://colab.research.google.com/github/Mark-Barbaric/IBM_Machine_Learning_Certificate/blob/dl_wee5/Deep_Learning_and_Reinforcement_Learning/week5/transfer_learning_with_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>


# **End-to-End Example: Transfer Learning**


Estimated time needed: **45** minutes


In this lab, we will walk through an end-to-end example of transfer learning and fine-tuning.


<h1> Is the waste product organic or a recyclable? </h1></s>


You are a data science intern at a waste management service. Your manager has asked you to create a waste classification pipeline that categorizes waste streams based on disposal options: organic or recyclable.

In this lab we're going to train a transfer learning model to perform this image classification task.


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module3/L2/img/transfer_learning.gif" width="600" height="600">


## **Table of Contents**

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
    </li>
    <li>
        <a href="#Background">Background</a>
        <ol>
            <li><a href="#What is transfer learning?">What is transfer learning?</a></li>
        </ol>
    </li>
        <li>
        <a href="#Example 1: Create a ML model for distinguishing recyclable and organic waste images">Example 1: Create a ML model for distinguishing recyclable and organic waste images</a>
            <ol>
            <li><a href="#Dataset">Dataset</a></li>
            <li><a href="#Importing data">Importing data</a></li>
            <li><a href="#Data visualization">Data visualization</a></li>
            <li><a href="#Define configuration options">Define configuration options</a></li>
            <li><a href="#Loading Images using ImageGeneratorClass">Loading Images using ImageGeneratorClass</a></li>
            <li><a href="#Pre-trained models">Pre-trained models</a></li>
            <li><a href="#Compile the model">Compile the model</a></li>
            <li><a href="#Fine-Tuning">Fine-Tuning</a></li>
            <li><a href="#Evaluate both models on test data">Evaluate both models on test data</a></li>
        </ol>
        </li>
    <li>
        <a href="#Example 2: Use Transfer Learning for identifying Stop Signs">Example 2: Use Transfer Learning for identifying Stop Signs</a>
            <ol>
            <li><a href="#Loading images">Loading images</a></li>
            <li><a href="#Defining a helper function">Defining a helper function</a></li>
            <li><a href="#Pre-trained Model 1: Incepton-v3">Pre-trained Model 1: Incepton-v3</a></li>
            <li><a href="#Pre-trained Model 2: MobileNet">Pre-trained Model 2: MobileNet</a></li>
            <li><a href="#Pre-trained Model 3: ResNet-50">Pre-trained Model 3: ResNet-50</a></li>
         </ol>
        </li>
</ol>


## Objectives

After completing this lab you will be able to:

- __Perform__ pre-processing and image augmentation on ImageGeneratorClass objects in Keras.
- __Implement__ transfer learning in five general steps:
    - obtain pre-trained model,
    - create base model,
    - freeze layers,
    - train new layers on dataset,
    - improve model through fine tuning.
- __Build__ an end-to-end VGG16-based transfer learning model for binary image classification tasks.


### Installing Required Libraries

The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run these notebook commands in a different Jupyter environment (like Watson Studio or Ananconda), you will need to install these libraries by removing the `#` sign before `!pip install tensorflow_datasets`, `!pip install --upgrade tensorflow` and `!pip install opendatasets` in the following code cell.


In [5]:
!pip install skillsnetwork

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 42.5 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [6]:
import numpy as np
import datetime
import os
import random, shutil
import glob
import skillsnetwork

import warnings
warnings.simplefilter('ignore')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib import pyplot
from matplotlib.image import imread

from os import makedirs,listdir
from shutil import copyfile
from random import seed
from random import random
import keras
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D, Input
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from sklearn import metrics


sns.set_context('notebook')
sns.set_style('white')

2.17.1


## Background


### What is transfer learning?


Most popular models are difficult to train from scratch as they require huge datasets (like ImageNet), a large number of training iterations, and very heavy computing machinery. The basic features (edges, shapes) learned by early layers in a network are generalizable. While the later layers in an already trained network tend to capture features that are more particular to a specific image classification task.

__Transfer learning__ uses the idea that if we keep the early layers of a pre-trained network, and re-train the later layers on a specific dataset, we might be able to leverage some state of that network on a related task.


A typical transfer learning workflow in Keras looks something like this:
    
1. Initialize base model, and load pre-trained weights (like ImageNet).
2. "Freeze" layers in the base model by setting `training = False`.
3. Define a new model that goes on top of the output of the base model's layers.
4. Train resulting model on your data set.


## Example 1: Create a ML model for distinguishing recyclable and organic waste images


### Dataset


The dataset used in this Lab was found on Kaggle, which is an awesome community for people who enjoy anything related to data science and machine learning. We will be using the Waste Classification Dataset [dataset](https://www.kaggle.com/datasets/techsash/waste-classification-data):


> PROBLEM:
> - Waste management is a big problem in our country. Most of the wastes end up in landfills.

> APPROACH:
> - Segregated into two classes (Organic and recyclable)
> - Automated the process by using IOT and machine learning

> IMPLEMENTATION:
> - Dataset is divided into train data (85%) and test data (15%)
> - Training data - 22,564 images Test data - 2,513 images


* The dataset contains ~25,000 images of recyclable and organic products split into a train and test set.
* Our goal is to train an algorithm on these files and to predict the labels for images in our test set (1 = recyclable, 0 = organic).


Now that we have obtained the necessary context and some insight in today's dataset, we're moving on to the practical part: using transfer learning to classify images. In this task, we will be using Tensorflow, which is a highly used ML library for training neural networks, and Keras, which is an API that makes this process simpler.


We will go through the following steps to create our classification model:


1. Import data directly from Kaggle.
2. Visualize a few random images from the train set.
3. Load images in using the ImageGeneratorClass from Keras.
4. Define model configuration options.
5. Perform some image augmentation for improved model generalizability.
6. Load, compile and train a pre-trained model like VGG-16.
7. Perform inference on the test set.


### Importing Data


This will create a `o-vs-r-split` directory in your environment.


In [7]:
import skillsnetwork
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/datasets/o-vs-r-split.tar.gz",
                           overwrite=True)


  0%|          | 0/8097 [00:00<?, ?it/s]

Saved to '.'


### Define configuration options


* __img_rows__ and __img_cols__ are used to specify width and height of the images expected by MobileNet.
* __batch size__ is set to 32.
* The __number of epochs__ (that is, iterations) is set to 5, but as we are using Early Stopping, the number of iterations might be lower if early stopping conditions are met before 5 iterations.
* The __number of classes__ is 2.
* We will use 20% of the data for __validation__ purposes.
* We want to be able to see the model output, so we set __verbosity__ to 1, or True.
* The __path__ is a path to the directory with the training data and __path_test__ should contain the test data.
* The __input_shape__ is common for an image: (w, h, d).
* We have two __labels__ in our dataset: organic (O), recyclable (R).
* The __checkpoint_path__ is where ModelCheckpoint will save our model.


In [8]:
img_rows, img_cols = 150, 150
batch_size = 32
n_epochs = 10
n_classes = 2
val_split = 0.2
verbosity = 1
path = 'o-vs-r-split/train/'
path_test = 'o-vs-r-split/test/'
input_shape = (img_rows, img_cols, 3) #RGB
labels = ['O', 'R']
seed = 10
checkpoint_path='ORnet.h5'

### Loading Images using ImageGeneratorClass


Transfer learning works best when models are trained on smaller datasets. We are using 10,000 images from the original dataset of 25,000 images. We are setting aside 10% of the set for testing purposes.


```python
o-vs-r-split/
└── train
    ├── O
    └── R
└── test
    ├── O
    └── R
```


#### Image Augmentation


Now we will create ImageDataGenerators used for training, validation, and testing.

Image data generators create batches of tensor image data with real-time data augmentation. They loop over the data in batches and are useful in feeding data to the training process. We specify a 20% validation split.

We will perform a few image augmentation steps. We set __rescale__ to 1./255. This means that each image pixel will be divided by 255 in order to normalize the image. We set the __width_shift_range__ and __height_shift_range__ to 0.1 each, and the __horizontal_flip__ to True.
* The __width_shift_range__ shifts the image horizontally (left or right).
* The __height_shift_range__ shifts the image vertically (up or down).
* The __horizontal_flip__ randomly flip inputs horizontally.


In [9]:
train_datagen = ImageDataGenerator(
    validation_split=val_split,
    rescale=1.0/255.0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    validation_split=val_split,
    rescale=1.0/255.0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

Since it is undesirable to load all of our image data into memory all at once, we make use of the __flow_from_directory__ method. It takes the ImageDataGenerator training/validation sets and flows image batches from a specified folder.

* We set __directory__ to the __path__, specifying the path to our training dataset.
* We specify classes using __labels__ we had defined in the configuration step.
* We apply a __seed__ so our random initializer is generated in the same way each time we perform this step. This allows for direct comparison between experiments.


In [10]:
train_generator = train_datagen.flow_from_directory(
    directory=path,
    classes=labels,
    seed=seed,
    batch_size=batch_size,
    shuffle=True,
    target_size=(img_rows, img_rows),
    subset="training"
)

Found 5810 images belonging to 2 classes.


In [12]:
val_generator = val_datagen.flow_from_directory(
    directory=path,
    classes=labels,
    seed=seed,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
    target_size=(img_rows, img_rows),
    subset="validation"
)

Found 1452 images belonging to 2 classes.


In [13]:
test_generator = test_datagen.flow_from_directory(
    directory=path_test,
    classes=labels,
    class_mode="binary",
    seed=seed,
    batch_size=batch_size,
    shuffle=True,
    target_size=(img_rows, img_rows)
)

Found 828 images belonging to 2 classes.
